In [1]:
import os
import sys
import numpy as np
import torch

sys.path.append("../")

from src.utils.constants import DATA_DIR, DEVICE
from src.train import train_model
from src.models import PhasePredictor
from src.datasets import PhaseDataset

In [2]:
device = torch.device(DEVICE)

In [3]:
labelled_data = np.load(os.path.join(DATA_DIR, "labelled_dataset.npz"))
unlabelled_data = np.load(os.path.join(DATA_DIR, "unlabelled_dataset.npz"))

In [4]:
X = labelled_data["X"]
Y = labelled_data["Y"]
qual_input_dims = labelled_data['X_qual_num_classes']

In [5]:
num_models = 10

models = [PhasePredictor(input_dim=X.shape[1], qual_input_dims=qual_input_dims, latent_dimension=64) for i in range(num_models)]
for model in models:
    model.to(device)
    train_model(model, X, Y, verbose=True)

/Users/junhalee/Desktop/perovskite-phase-pred/venv39/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 0: Train Loss: 0.6822824861322131, Test Loss: 0.7322649359703064
Epoch 30: Train Loss: 0.3456585162452289, Test Loss: 0.38584619760513306
Epoch 60: Train Loss: 0.21739506162703037, Test Loss: 0.3940729796886444
Epoch 90: Train Loss: 0.2282195926776954, Test Loss: 0.35482170805335045
Epoch 120: Train Loss: 0.16089871472546033, Test Loss: 0.3575807362794876
Epoch 150: Train Loss: 0.15791106889290468, Test Loss: 0.3570442870259285
Epoch 180: Train Loss: 0.1498711651989392, Test Loss: 0.3572389744222164
Epoch 210: Train Loss: 0.15520719970975602, Test Loss: 0.35730184614658356
Epoch 240: Train Loss: 0.1876509359904698, Test Loss: 0.3573109284043312
Epoch 270: Train Loss: 0.19752335122653417, Test Loss: 0.35732633620500565
Epoch 300: Train Loss: 0.1872111734535013, Test Loss: 0.35733501613140106
Epoch 330: Train Loss: 0.21292401903441974, Test Loss: 0.3573480136692524
Epoch 360: Train Loss: 0.17260902641075, Test Loss: 0.35735470801591873
Epoch 390: Train Loss: 0.14379279288862432, Te

In [6]:
def inference(X):
    predictions = []
    for model in models:
        pred = model(X)
        predictions.append(pred)
    
    # Get the mean and variance of the predictions
    predictions = torch.stack(predictions)
    mean = predictions.mean(dim=0)
    var = predictions.var(dim=0)

    return mean, var

In [7]:
print(unlabelled_data['X'].shape)

(1664, 37)


In [8]:
X = torch.tensor(unlabelled_data['X'], device=device, requires_grad=False).float()

In [10]:
mean, var = inference(X)

In [11]:
print(mean.max(), var.max())

tensor(0.9969, grad_fn=<MaxBackward1>) tensor(0.0765, grad_fn=<MaxBackward1>)
